#### Install ctmatch from repo, all dependencies

In [ ]:
!git clone https://github.com/semajyllek/ctmatch.git

Cloning into 'ctmatch'...
remote: Enumerating objects: 1830, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 1830 (delta 189), reused 268 (delta 167), pack-reused 1540
Receiving objects: 100% (1830/1830), 550.07 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (1226/1226), done.


In [ ]:
!pip install -r /content/ctmatch/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0

In [ ]:
from huggingface_hub import notebook_login    

notebook_login()

In [ ]:
import sys
sys.path.insert(1, '/content/ctmatch')

In [ ]:
from ctmatch.match import CTMatch, PipeConfig

Evaluate the pre-pruned model for a baseline.

In [ ]:
pipe_config = PipeConfig(
    prune=False, 
    classifier_model_checkpoint='semaj83/scibert_finetuned_ctmatch'
)
ctm = CTMatch(pipe_config)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/semaj83___json/semaj83--ctmatch_classification-c5ae6b4cf5476186/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

In [ ]:
ctm.classifier_model.dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28576
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7939
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3176
    })
})

In [ ]:
from time import perf_counter
start = perf_counter()
cm, cr = ctm.classifier_model.get_sklearn_metrics()
end = perf_counter()

print(f"total elapsed time: {end - start}")

/content/ctmatch/ctmatch/models/classifier_model.py:258: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).unsqueeze(0).to(self.device)


total elapsed time: 101.07220333200002


In [ ]:
print(cr)

              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5430
           1       0.44      0.63      0.52      1331
           2       0.53      0.47      0.50      1178

    accuracy                           0.74      7939
   macro avg       0.62      0.64      0.63      7939
weighted avg       0.77      0.74      0.75      7939



In [ ]:
unpruned_model = ctm.classifier_model.model

Prune and evalute the pruned model, save it to the hub.

In [ ]:
pipe_config = PipeConfig(
    classifier_model_checkpoint='semaj83/scibert_finetuned_pruned_ctmatch',
)
ctm = CTMatch(pipe_config)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

removed heads 0, total_heads=129, percentage removed=0.0
bert.encoder.layer.0.intermediate.dense, sparsity = 50.00
bert.encoder.layer.0.output.dense, sparsity = 50.00
bert.encoder.layer.1.intermediate.dense, sparsity = 50.00
bert.encoder.layer.1.output.dense, sparsity = 50.00
bert.encoder.layer.2.intermediate.dense, sparsity = 50.00
bert.encoder.layer.2.output.dense, sparsity = 50.00
bert.encoder.layer.3.intermediate.dense, sparsity = 50.00
bert.encoder.layer.3.output.dense, sparsity = 50.00
bert.encoder.layer.4.intermediate.dense, sparsity = 50.00
bert.encoder.layer.4.output.dense, sparsity = 50.00
bert.encoder.layer.5.intermediate.dense, sparsity = 50.00
bert.encoder.layer.5.output.dense, sparsity = 50.00
bert.encoder.layer.6.intermediate.dense, sparsity = 50.00
bert.encoder.layer.6.output.dense, sparsity = 50.00
bert.encoder.layer.7.intermediate.dense, sparsity = 50.00
bert.encoder.layer.7.output.dense, sparsity = 50.00
bert.encoder.layer.8.intermediate.dense, sparsity = 50.00
bert.

In [ ]:
#ctm.classifier_model.prune_model()

In [ ]:
from time import perf_counter
start = perf_counter()
cm, cr = ctm.classifier_model.get_sklearn_metrics()
end = perf_counter()

print(f"total elapsed time: {end - start}")

/content/ctmatch/ctmatch/models/classifier_model.py:258: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).unsqueeze(0).to(self.device)


total elapsed time: 91.06503826899996


In [ ]:
print(cr)

              precision    recall  f1-score   support

           0       0.85      0.92      0.88      5430
           1       0.60      0.37      0.46      1331
           2       0.50      0.53      0.51      1178

    accuracy                           0.77      7939
   macro avg       0.65      0.61      0.62      7939
weighted avg       0.76      0.77      0.76      7939



In [ ]:
ctm.classifier_model.model.num_parameters() / unpruned_model.num_parameters()

0.7154133498572349